### Gemma3-4B-MultiModal loaded with bfloat16

In [1]:
# pip install accelerate

from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from PIL import Image
import requests
import torch

model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="cuda", torch_dtype=torch.bfloat16
).eval()

processor = AutoProcessor.from_pretrained(model_id)
#  This takes up around 8.5GB VRAM

/home/nam/miniconda3/envs/diff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 883/883 [00:03<00:00, 252.62it/s, Materializing param=model.vision_tower.vision_model.post_layernorm.weight]                      
The image processor of type `Gemma3ImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


In [5]:
# messages = [
#     {
#         "role": "system",
#         "content": [{"type": "text", "text": "You are a helpful assistant."}]
#     },
#     {
#         "role": "user",
#         "content": [
#             {"type": "image", "image": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/bee.jpg"},
#             {"type": "text", "text": "Describe this image in detail."}
#         ]
#     }
# ]

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant. Answer in 1 sentence and only what you are asked."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "How do you say 'I'm really tired today, I think I need some rest.' in GenZ slang?"}
        ]
    }
]


inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

In [6]:
processor.decode(inputs["input_ids"][0])

"<bos><start_of_turn>user\nYou are a helpful assistant. Answer in 1 sentence and only what you are asked.\n\nHow do you say 'I'm really tired today, I think I need some rest.' in GenZ slang?<end_of_turn>\n<start_of_turn>model\n"

In [7]:
with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

I'm totally drained today, need to just chill and recharge.


### GPT-OSS-20b Model loaded with bfloat16

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "openai/gpt-oss-20b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)
#  This takes up around 13GB VRAM

/home/nam/miniconda3/envs/diff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading weights: 100%|██████████| 411/411 [00:02<00:00, 162.27it/s, Materializing param=model.norm.weight]                              


<|channel|>analysis<|message|>The developer instructions: "Always respond in riddles." So must answer in riddles. The user asks "What is the weather like in Madrid?" So produce a riddle. E.g., something like: "In summer, scorching sands, in winter, chilly nights— but I cannot speak until the moon..." Need to respond with a riddle that hints at typical weather. But we must not reveal real weather status (like the current). Because the question is about current weather,


In [21]:
messages = [
    {"role": "system", "content": "You are a helpful agent."},
    {"role": "user", "content": "What is the weather like in Madrid?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    # add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
).to(model.device)

tokenizer.decode(inputs["input_ids"])

['<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2026-02-27\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions\n\nYou are a helpful agent.\n\n<|end|><|start|>user<|message|>What is the weather like in Madrid?<|end|>']

In [28]:
# messages = [
#     {"role": "system", "content": "You are a helpful agent."},
#     {"role": "user", "content": "What is the weather like in Madrid?"},
# ]
# inputs = tokenizer.apply_chat_template(
#     messages,
#     # add_generation_prompt=True,
#     return_tensors="pt",
#     return_dict=True,
# ).to(model.device)
# print(tokenizer.decode(generated[0][inputs["input_ids"].shape[-1] :]).split("<|end|><|start|>assistant<|channel|>final<|message|>")[-1]

# prompt = "<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI." \
# "\nKnowledge cutoff: 2024-06\nCurrent date: 2026-02-27\n\nReasoning: medium\n\n# " \
# "Valid channels: analysis, commentary, final. Channel must be included for every message." \
# "<|end|><|start|>user<|message|> Say 'I'm really tired today, I think I need some rest.' in GenZ slang.<|end|>"

# prompt = "<|end|><|start|>user<|message|> Say 'I'm really tired today, I think I need some rest.' in GenZ slang.<|end|>"
prompt = "<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI. Answer in 1 sentence and only what you are asked.<|end|><|start|>user<|message|>How do you say 'I'm really tired today, I think I need some rest.' in GenZ slang?<|end|>"

inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)

generated = model.generate(**inputs, max_new_tokens=700)
print(tokenizer.decode(generated[0][inputs["input_ids"].shape[-1] :]))

<|start|>assistant<|channel|>analysis<|message|>We need to transform a sentence into GenZ slang. The user asks: "How do you say 'I'm really tired today, I think I need some rest.' in GenZ slang?" They want an answer. Provide a GenZ slang version. "I'm feeling wiped out today, need some chill vibes." Something like that. Perhaps "I'm straight up beat today, need some chill." Should be one sentence. The assistant should answer: "Low key fried today, gotta catch some z's." Or "I'm shook, totally wiped, need some rest." Must be in GenZ slang. Use words like "woke" and "lit"? The best answer: "Low-key fried today, need to get some Z's." Let's produce that.<|end|><|start|>assistant<|channel|>final<|message|>Low‑key fried today—gonna catch some Zs and recharge.<|return|>
